In [6]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
# Install kaggle package
!pip install kaggle

# Import libraries
import sqlite3
import pandas as pd
import os

# Download Kaggle dataset
try:
    !kaggle datasets download -d vijayuv/onlineretail
    !unzip onlineretail.zip
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Ensure Kaggle API is configured and internet is active.")
    raise

# Connect to SQLite database (in-memory for Colab)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load data into pandas
try:
    df = pd.read_csv('OnlineRetail.csv', encoding='latin1')
    # Sample 20,000 rows to capture more customer transactions
    df = df.sample(n=20000, random_state=42)
except FileNotFoundError:
    print("Error: OnlineRetail.csv not found. Check Kaggle download.")
    raise
except Exception as e:
    print(f"Error loading CSV file: {e}")
    raise

# Create SQLite table
cursor.execute('''
CREATE TABLE Retail (
    InvoiceNo TEXT,
    StockCode TEXT,
    Description TEXT,
    Quantity INTEGER,
    InvoiceDate TEXT,
    UnitPrice REAL,
    CustomerID INTEGER,
    Country TEXT
)
''')

# Insert data into SQLite
df.to_sql('Retail', conn, if_exists='append', index=False)

# SQL Queries to demonstrate skills and provide insights
queries = [
    # Query 1: Total revenue by country
    '''
    SELECT
        Country,
        ROUND(SUM(Quantity * UnitPrice), 2) as total_revenue
    FROM Retail
    WHERE CustomerID IS NOT NULL
    GROUP BY Country
    ORDER BY total_revenue DESC
    LIMIT 5;
    ''',

    # Query 2: Top products by units sold
    '''
    SELECT
        Description,
        SUM(Quantity) as total_units_sold,
        ROUND(SUM(Quantity * UnitPrice), 2) as total_revenue
    FROM Retail
    WHERE Quantity > 0
    GROUP BY Description
    ORDER BY total_units_sold DESC
    LIMIT 5;
    ''',

    # Query 3: High-value customers (above-average spend)
    '''
    WITH CustomerSpend AS (
        SELECT
            CustomerID,
            ROUND(SUM(Quantity * UnitPrice), 2) as total_spent
        FROM Retail
        WHERE CustomerID IS NOT NULL
        GROUP BY CustomerID
    )
    SELECT
        CustomerID,
        total_spent
    FROM CustomerSpend
    WHERE total_spent > (SELECT AVG(total_spent) FROM CustomerSpend)
    ORDER BY total_spent DESC
    LIMIT 5;
    ''',

    # Query 4: Average order value by country
    '''
    WITH OrderValues AS (
        SELECT
            InvoiceNo,
            Country,
            ROUND(SUM(Quantity * UnitPrice), 2) as order_value
        FROM Retail
        WHERE CustomerID IS NOT NULL
        GROUP BY InvoiceNo, Country
    )
    SELECT
        Country,
        ROUND(AVG(order_value), 2) as avg_order_value
    FROM OrderValues
    GROUP BY Country
    ORDER BY avg_order_value DESC
    LIMIT 5;
    ''',

    # Query 5: Product category profitability (based on Description keywords)
    '''
    SELECT
        CASE
            WHEN Description LIKE '%LIGHT%' OR Description LIKE '%LANTERN%' THEN 'Home Decor'
            WHEN Description LIKE '%BAG%' OR Description LIKE '%CASE%' THEN 'Accessories'
            ELSE 'Other'
        END as product_category,
        COUNT(DISTINCT Description) as unique_products,
        ROUND(SUM(Quantity * UnitPrice), 2) as category_revenue
    FROM Retail
    WHERE Quantity > 0 AND CustomerID IS NOT NULL
    GROUP BY product_category
    ORDER BY category_revenue DESC;
    ''',

    # Query 6: Top repeat customers by invoice count
    '''
    SELECT
        CustomerID,
        COUNT(DISTINCT InvoiceNo) as purchase_count,
        ROUND(SUM(Quantity * UnitPrice), 2) as total_spent
    FROM Retail
    WHERE CustomerID IS NOT NULL
    GROUP BY CustomerID
    HAVING purchase_count > 1
    ORDER BY purchase_count DESC, total_spent DESC
    LIMIT 5;
    '''
]

# Execute and display results
for i, query in enumerate(queries, 1):
    print(f"\nQuery {i} Results:")
    try:
        df_result = pd.read_sql_query(query, conn)
        print(df_result)
    except Exception as e:
        print(f"Error executing query {i}: {e}")

# Clean up
try:
    os.remove('onlineretail.zip')
    os.remove('OnlineRetail.csv')
except:
    pass
conn.close()

Dataset URL: https://www.kaggle.com/datasets/vijayuv/onlineretail
License(s): CC0-1.0
Archive:  onlineretail.zip
  inflating: OnlineRetail.csv        

Query 1 Results:
          Country  total_revenue
0  United Kingdom      242094.80
1     Netherlands       11394.20
2            EIRE       11061.86
3         Germany        8170.93
4          France        7198.34

Query 2 Results:
                         Description  total_units_sold  total_revenue
0  WORLD WAR 2 GLIDERS ASSTD DESIGNS              5382        1083.54
1          ASSORTED COLOURS SILK FAN              2714        1842.30
2        GIN + TONIC DIET METAL SIGN              2351        4090.29
3            JUMBO BAG RED RETROSPOT              2345        4465.05
4    PACK OF 72 RETROSPOT CAKE CASES              2231        1134.75

Query 3 Results:
   CustomerID  total_spent
0       14646     11109.03
1       17450      6413.94
2       14156      5475.62
3       14911      5457.88
4       18102      4909.15

Query 4 Result